# Pixel Amplifier Testing & Calibration via Guard Ring

In order to test the Charge Sensitive Amplifiers (CSA) within the pixels, we need to inject charge into the inputs of the amplifiers. We do this using the parasitic capacitance of the guard ring structure, which surrounds the topmetal input electrodes of the pixels. By injecting a known square wave pulse into the guard ring, we can look at the corresponding pixel response to calibrate our amplifiers.

Let's take a look at this by selecting out a single pixel from the array (which we do by stopping the clock on that pixel) and injecting a pulse.